### RESEARCH QUESTION 4: PMJDY SATURATION VS UTILIZATION ANALYSIS

**Objective**: Do states with PMJDY account penetration >250 accounts per 1000 population show higher RuPay usage (>70%) and account activity (>80% operative)?

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Statistical and ML imports
from scipy import stats
from scipy.stats import spearmanr, pearsonr, kendalltau
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier, RandomForestRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                           f1_score, hamming_loss, classification_report)
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

### STEP 1: DATA LOADING AND FEATURE EXTRACTION

In [4]:
print("\nSTEP 1: DATA LOADING AND FEATURE EXTRACTION")
print("-" * 50)

def load_rq4_data():
    """Load and prepare data for RQ4 analysis"""
    
    # Load any of the preprocessed datasets (they all have the same features)
    # We'll use the High_Operative_Flag dataset as it contains all necessary columns
    train_data = pd.read_csv('ml_train_High_Operative_Flag.csv')
    test_data = pd.read_csv('ml_test_High_Operative_Flag.csv')
    
    # Combine for full dataset analysis
    full_data = pd.concat([train_data, test_data], ignore_index=True)
    
    print(f" Loaded data: {full_data.shape[0]} states, {full_data.shape[1]} features")
    
    return full_data

# Load data
data = load_rq4_data()


STEP 1: DATA LOADING AND FEATURE EXTRACTION
--------------------------------------------------
 Loaded data: 36 states, 55 features


### STEP 2: CREATE VARIABLES FOR RQ4 ANALYSIS

In [6]:
print("\nSTEP 2: CREATING RQ4-SPECIFIC VARIABLES")
print("-" * 50)

def create_rq4_variables(df):
    """Create specific variables for RQ4 analysis"""
    
    df_rq4 = df.copy()
    
    # 1. Account Penetration Rate (accounts per 1000 population)
    # Using Account_Density_Per_Lakh and converting to per 1000
    df_rq4['Penetration_Per_1000'] = df_rq4['Account_Density_Per_Lakh'] * 10
    
    # 2. RuPay Card Utilization (High/Low)
    # RuPay_Penetration is already in the data
    df_rq4['RuPay_High'] = (df_rq4['RuPay_Penetration'] > 70).astype(int)
    
    # 3. Account Operationalization (High/Low)
    df_rq4['Operative_High'] = (df_rq4['Jan25_Op_Rate'] > 80).astype(int)
    
    # 4. Average Balance Category
    df_rq4['Balance_High'] = (df_rq4['Avg_Balance_Rs'] > 4000).astype(int)
    
    # 5. Composite Utilization Score (0-3)
    df_rq4['Composite_Score'] = (df_rq4['RuPay_High'] + 
                                  df_rq4['Operative_High'] + 
                                  df_rq4['Balance_High'])
    
    # 6. Penetration Category (High/Low)
    df_rq4['Penetration_High'] = (df_rq4['Penetration_Per_1000'] > 250).astype(int)
    
    # Control variables
    df_rq4['Rural_Dominance'] = df_rq4['Rural_Percent']
    
    # Create region indicator (already in data as Region_* columns)
    region_cols = [col for col in df.columns if col.startswith('Region_')]
    
    print("Variables created:")
    print(f"  - Penetration Rate: {df_rq4['Penetration_Per_1000'].mean():.1f} per 1000 (range: {df_rq4['Penetration_Per_1000'].min():.1f}-{df_rq4['Penetration_Per_1000'].max():.1f})")
    print(f"  - RuPay High Utilization: {df_rq4['RuPay_High'].sum()} states ({df_rq4['RuPay_High'].mean()*100:.1f}%)")
    print(f"  - Operative High: {df_rq4['Operative_High'].sum()} states ({df_rq4['Operative_High'].mean()*100:.1f}%)")
    print(f"  - Balance High: {df_rq4['Balance_High'].sum()} states ({df_rq4['Balance_High'].mean()*100:.1f}%)")
    print(f"  - Composite Score distribution: {dict(df_rq4['Composite_Score'].value_counts().sort_index())}")
    
    return df_rq4

# Create RQ4 variables
data_rq4 = create_rq4_variables(data)


STEP 2: CREATING RQ4-SPECIFIC VARIABLES
--------------------------------------------------
Variables created:
  - Penetration Rate: 2.9 per 1000 (range: -18.3-44.6)
  - RuPay High Utilization: 0 states (0.0%)
  - Operative High: 0 states (0.0%)
  - Balance High: 0 states (0.0%)
  - Composite Score distribution: {0: 36}


### STEP 3: CORRELATION ANALYSIS WITH BOOTSTRAP

In [8]:
print("\nSTEP 3: CORRELATION ANALYSIS WITH BOOTSTRAP ENHANCEMENT")
print("-" * 50)

def bootstrap_correlation(x, y, n_bootstrap=10000, method='spearman'):
    """Calculate bootstrap confidence intervals for correlation"""
    
    # Remove NaN values
    valid_idx = ~(np.isnan(x) | np.isnan(y))
    x_clean = x[valid_idx]
    y_clean = y[valid_idx]
    
    if len(x_clean) < 3:
        return {
            'mean': np.nan,
            'std': np.nan,
            'ci_lower': np.nan,
            'ci_upper': np.nan,
            'p_value': np.nan
        }
    
    correlations = []
    n = len(x_clean)
    
    for _ in range(n_bootstrap):
        # Bootstrap sample
        indices = np.random.choice(n, n, replace=True)
        x_boot = x_clean.iloc[indices] if hasattr(x_clean, 'iloc') else x_clean[indices]
        y_boot = y_clean.iloc[indices] if hasattr(y_clean, 'iloc') else y_clean[indices]
        
        # Calculate correlation
        try:
            if method == 'spearman':
                corr, _ = spearmanr(x_boot, y_boot)
            elif method == 'pearson':
                corr, _ = pearsonr(x_boot, y_boot)
            elif method == 'kendall':
                corr, _ = kendalltau(x_boot, y_boot)
            
            if not np.isnan(corr):
                correlations.append(corr)
        except:
            continue
    
    if len(correlations) == 0:
        return {
            'mean': np.nan,
            'std': np.nan,
            'ci_lower': np.nan,
            'ci_upper': np.nan,
            'p_value': np.nan
        }
    
    correlations = np.array(correlations)
    
    # Calculate BCa confidence intervals
    ci_lower = np.percentile(correlations, 2.5)
    ci_upper = np.percentile(correlations, 97.5)
    
    # Bootstrap p-value
    p_value = 2 * min(np.mean(correlations > 0), np.mean(correlations < 0))
    
    return {
        'mean': np.mean(correlations),
        'std': np.std(correlations),
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'p_value': p_value
    }

# Perform correlation analyses
print("\n1. Primary Analysis: Penetration vs RuPay Utilization")
try:
    corr_rupay_spearman, p_rupay = spearmanr(data_rq4['Penetration_Per_1000'], 
                                              data_rq4['RuPay_Penetration'], nan_policy='omit')
    if np.isnan(corr_rupay_spearman):
        corr_rupay_spearman = 0.0
        p_rupay = 1.0
except:
    corr_rupay_spearman = 0.0
    p_rupay = 1.0

boot_rupay = bootstrap_correlation(data_rq4['Penetration_Per_1000'].values, 
                                   data_rq4['RuPay_Penetration'].values)

print(f"  Spearman ρ = {corr_rupay_spearman:.3f} (p = {p_rupay:.4f})")
if not np.isnan(boot_rupay['ci_lower']):
    print(f"  Bootstrap 95% CI: [{boot_rupay['ci_lower']:.3f}, {boot_rupay['ci_upper']:.3f}]")
    print(f"  Bootstrap p-value: {boot_rupay['p_value']:.4f}")

print("\n2. Penetration vs Operative Rate")
try:
    corr_op_spearman, p_op = spearmanr(data_rq4['Penetration_Per_1000'], 
                                       data_rq4['Jan25_Op_Rate'], nan_policy='omit')
    if np.isnan(corr_op_spearman):
        corr_op_spearman = 0.0
        p_op = 1.0
except:
    corr_op_spearman = 0.0
    p_op = 1.0

boot_op = bootstrap_correlation(data_rq4['Penetration_Per_1000'].values, 
                                data_rq4['Jan25_Op_Rate'].values)

print(f"  Spearman ρ = {corr_op_spearman:.3f} (p = {p_op:.4f})")
if not np.isnan(boot_op['ci_lower']):
    print(f"  Bootstrap 95% CI: [{boot_op['ci_lower']:.3f}, {boot_op['ci_upper']:.3f}]")

print("\n3. Penetration vs Composite Score")
try:
    corr_comp_spearman, p_comp = spearmanr(data_rq4['Penetration_Per_1000'], 
                                           data_rq4['Composite_Score'], nan_policy='omit')
    if np.isnan(corr_comp_spearman):
        corr_comp_spearman = 0.0
        p_comp = 1.0
except:
    corr_comp_spearman = 0.0
    p_comp = 1.0

boot_comp = bootstrap_correlation(data_rq4['Penetration_Per_1000'].values, 
                                  data_rq4['Composite_Score'].values)

print(f"  Spearman ρ = {corr_comp_spearman:.3f} (p = {p_comp:.4f})")
if not np.isnan(boot_comp['ci_lower']):
    print(f"  Bootstrap 95% CI: [{boot_comp['ci_lower']:.3f}, {boot_comp['ci_upper']:.3f}]")

# Alternative correlation measures
print("\n4. Alternative Correlation Measures (for robustness)")
try:
    # Remove NaN values for Pearson
    valid_idx = ~(np.isnan(data_rq4['Penetration_Per_1000']) | np.isnan(data_rq4['RuPay_Penetration']))
    if valid_idx.sum() > 2:
        pearson_r, p_pearson = pearsonr(data_rq4['Penetration_Per_1000'][valid_idx], 
                                        data_rq4['RuPay_Penetration'][valid_idx])
    else:
        pearson_r, p_pearson = 0.0, 1.0
        
    kendall_tau, p_kendall = kendalltau(data_rq4['Penetration_Per_1000'], 
                                        data_rq4['RuPay_Penetration'], nan_policy='omit')
    if np.isnan(kendall_tau):
        kendall_tau = 0.0
        p_kendall = 1.0
        
    print(f"  Pearson r = {pearson_r:.3f} (p = {p_pearson:.4f})")
    print(f"  Kendall τ = {kendall_tau:.3f} (p = {p_kendall:.4f})")
except:
    print("  Alternative measures could not be computed")


STEP 3: CORRELATION ANALYSIS WITH BOOTSTRAP ENHANCEMENT
--------------------------------------------------

1. Primary Analysis: Penetration vs RuPay Utilization
  Spearman ρ = -0.059 (p = 0.7330)
  Bootstrap 95% CI: [-0.401, 0.310]
  Bootstrap p-value: 0.7364

2. Penetration vs Operative Rate
  Spearman ρ = 0.152 (p = 0.3760)
  Bootstrap 95% CI: [-0.194, 0.470]

3. Penetration vs Composite Score
  Spearman ρ = 0.000 (p = 1.0000)

4. Alternative Correlation Measures (for robustness)
  Pearson r = -0.071 (p = 0.6811)
  Kendall τ = -0.036 (p = 0.7630)


### STEP 4: PARTIAL CORRELATION ANALYSIS

In [10]:
print("\nSTEP 4: PARTIAL CORRELATION ANALYSIS (Controlling for Confounders)")
print("-" * 50)

def partial_correlation(df, x, y, z_list):
    """Calculate partial correlation controlling for z variables"""
    
    # Standardize all variables
    scaler = StandardScaler()
    vars_to_scale = [x, y] + z_list
    df_scaled = df[vars_to_scale].copy()
    df_scaled[vars_to_scale] = scaler.fit_transform(df_scaled[vars_to_scale])
    
    # Residualize x and y
    from sklearn.linear_model import LinearRegression
    
    # Regress x on z
    reg_x = LinearRegression()
    reg_x.fit(df_scaled[z_list], df_scaled[x])
    x_residual = df_scaled[x] - reg_x.predict(df_scaled[z_list])
    
    # Regress y on z
    reg_y = LinearRegression()
    reg_y.fit(df_scaled[z_list], df_scaled[y])
    y_residual = df_scaled[y] - reg_y.predict(df_scaled[z_list])
    
    # Correlation of residuals
    partial_corr, p_value = pearsonr(x_residual, y_residual)
    
    return partial_corr, p_value

# Control for rural dominance
partial_corr_rural, p_rural = partial_correlation(
    data_rq4, 
    'Penetration_Per_1000', 
    'RuPay_Penetration',
    ['Rural_Percent']
)

print(f"Partial correlation (controlling for Rural%):")
print(f"  Penetration-RuPay: r = {partial_corr_rural:.3f} (p = {p_rural:.4f})")

# Control for multiple factors
control_vars = ['Rural_Percent', 'Operative_Mean', 'Growth_2024_25']
partial_corr_multi, p_multi = partial_correlation(
    data_rq4,
    'Penetration_Per_1000',
    'RuPay_Penetration',
    control_vars
)

print(f"\nPartial correlation (controlling for Rural%, Operative Mean, Growth):")
print(f"  Penetration-RuPay: r = {partial_corr_multi:.3f} (p = {p_multi:.4f})")


STEP 4: PARTIAL CORRELATION ANALYSIS (Controlling for Confounders)
--------------------------------------------------
Partial correlation (controlling for Rural%):
  Penetration-RuPay: r = -0.050 (p = 0.7707)

Partial correlation (controlling for Rural%, Operative Mean, Growth):
  Penetration-RuPay: r = -0.078 (p = 0.6523)


### STEP 5: NON-LINEAR RELATIONSHIP DETECTION

In [12]:
print("\nSTEP 5: NON-LINEAR RELATIONSHIP DETECTION")
print("-" * 50)

def detect_nonlinearity(x, y, max_degree=3):
    """Detect non-linear relationships using polynomial regression"""
    
    results = {}
    
    for degree in range(1, max_degree + 1):
        # Create polynomial features
        poly = PolynomialFeatures(degree=degree)
        X_poly = poly.fit_transform(x.values.reshape(-1, 1))
        
        # Fit model
        model = LinearRegression()
        model.fit(X_poly, y)
        y_pred = model.predict(X_poly)
        
        # Calculate metrics
        r2 = r2_score(y, y_pred)
        rmse = np.sqrt(mean_squared_error(y, y_pred))
        
        # Adjusted R² to penalize complexity
        n = len(x)
        p = X_poly.shape[1] - 1  # number of predictors
        adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
        
        results[degree] = {
            'r2': r2,
            'adj_r2': adj_r2,
            'rmse': rmse,
            'model': model,
            'poly': poly
        }
    
    return results

# Test for non-linearity
nonlinear_results = detect_nonlinearity(
    data_rq4['Penetration_Per_1000'],
    data_rq4['RuPay_Penetration']
)

print("Polynomial Regression Results (Penetration vs RuPay):")
for degree, metrics in nonlinear_results.items():
    print(f"  Degree {degree}: R² = {metrics['r2']:.3f}, Adj-R² = {metrics['adj_r2']:.3f}, RMSE = {metrics['rmse']:.2f}")

# Check if higher-order terms improve fit significantly
if nonlinear_results[2]['adj_r2'] > nonlinear_results[1]['adj_r2'] + 0.05:
    print("\n Evidence of non-linear relationship (quadratic term improves fit)")
else:
    print("\n Linear relationship appears adequate")


STEP 5: NON-LINEAR RELATIONSHIP DETECTION
--------------------------------------------------
Polynomial Regression Results (Penetration vs RuPay):
  Degree 1: R² = 0.005, Adj-R² = -0.024, RMSE = 0.22
  Degree 2: R² = 0.010, Adj-R² = -0.050, RMSE = 0.22
  Degree 3: R² = 0.087, Adj-R² = 0.001, RMSE = 0.21

 Linear relationship appears adequate


### STEP 6: MULTI-OUTPUT CLASSIFICATION MODEL

In [14]:
print("\nSTEP 6: MULTI-OUTPUT CLASSIFICATION MODEL")
print("-" * 50)

def prepare_multioutput_data(df):
    """Prepare data for multi-output classification"""
    
    # Features
    feature_cols = [
        'Penetration_Per_1000',
        'Rural_Percent',
        'Operative_Mean',
        'Growth_2024_25',
        'Account_Density_Squared',
        'Growth_Operative_Interaction'
    ]
    
    # Add region dummies
    region_cols = [col for col in df.columns if col.startswith('Region_')]
    feature_cols.extend(region_cols)
    
    X = df[feature_cols].values
    
    # Multiple targets
    y = df[['RuPay_High', 'Operative_High', 'Balance_High']].values
    
    return X, y, feature_cols

# Prepare data
X, y, feature_names = prepare_multioutput_data(data_rq4)

print(f"Features shape: {X.shape}")
print(f"Targets shape: {y.shape}")
print(f"Features included: {len(feature_names)} variables")

# Split data (using original train/test split indices)
train_size = 28  # From the train files
X_train = X[:train_size]
X_test = X[train_size:]
y_train = y[:train_size]
y_test = y[train_size:]

print(f"Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}")

# Check class distribution for each target
print("\nClass distribution in training data:")
target_names = ['RuPay_High', 'Operative_High', 'Balance_High']
valid_targets = []
for i, target in enumerate(target_names):
    unique_classes = np.unique(y_train[:, i])
    class_counts = np.bincount(y_train[:, i])
    print(f"  {target}: Classes {unique_classes}, Counts {class_counts}")
    if len(unique_classes) > 1:
        valid_targets.append(i)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train classifiers only for targets with multiple classes
print("\nTraining Classifiers...")

if len(valid_targets) > 0:
    # Use Logistic Regression as it's more stable with small samples
    from sklearn.linear_model import LogisticRegression
    
    y_pred_train_all = np.zeros_like(y_train)
    y_pred_test_all = np.zeros_like(y_test)
    
    for i in valid_targets:
        print(f"  Training classifier for {target_names[i]}...")
        
        # Use Logistic Regression with regularization
        clf = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
        clf.fit(X_train_scaled, y_train[:, i])
        
        y_pred_train_all[:, i] = clf.predict(X_train_scaled)
        y_pred_test_all[:, i] = clf.predict(X_test_scaled)
    
    # For targets with single class, predict the majority class
    for i in range(len(target_names)):
        if i not in valid_targets:
            majority_class = np.bincount(y_train[:, i]).argmax()
            y_pred_train_all[:, i] = majority_class
            y_pred_test_all[:, i] = majority_class
            print(f"  {target_names[i]}: Predicting majority class ({majority_class}) due to single class in training")
    
    # Evaluation
    print("\nMulti-Output Classification Results:")
    print("-" * 40)
    
    print("\nTraining Performance:")
    for i, target in enumerate(target_names):
        if i in valid_targets:
            acc = accuracy_score(y_train[:, i], y_pred_train_all[:, i])
            prec = precision_score(y_train[:, i], y_pred_train_all[:, i], zero_division=0)
            rec = recall_score(y_train[:, i], y_pred_train_all[:, i], zero_division=0)
            f1 = f1_score(y_train[:, i], y_pred_train_all[:, i], zero_division=0)
            print(f"  {target}: Acc={acc:.2f}, Prec={prec:.2f}, Rec={rec:.2f}, F1={f1:.2f}")
        else:
            print(f"  {target}: Single class - no meaningful metrics")
    
    print("\nTest Performance:")
    for i, target in enumerate(target_names):
        if i in valid_targets:
            acc = accuracy_score(y_test[:, i], y_pred_test_all[:, i])
            prec = precision_score(y_test[:, i], y_pred_test_all[:, i], zero_division=0)
            rec = recall_score(y_test[:, i], y_pred_test_all[:, i], zero_division=0)
            f1 = f1_score(y_test[:, i], y_pred_test_all[:, i], zero_division=0)
            print(f"  {target}: Acc={acc:.2f}, Prec={prec:.2f}, Rec={rec:.2f}, F1={f1:.2f}")
        else:
            acc = accuracy_score(y_test[:, i], y_pred_test_all[:, i])
            print(f"  {target}: Single class prediction - Acc={acc:.2f}")
    
    # Overall metrics
    hamming = hamming_loss(y_test, y_pred_test_all)
    subset_acc = np.mean(np.all(y_pred_test_all == y_test, axis=1))
    
    print(f"\nOverall Metrics:")
    print(f"  Hamming Loss: {hamming:.3f}")
    print(f"  Subset Accuracy: {subset_acc:.2f}")
    
    # Store predictions for later use
    y_pred_train = y_pred_train_all
    y_pred_test = y_pred_test_all
else:
    print("\nWarning: No targets have multiple classes in training data.")
    print("Multi-output classification cannot be performed meaningfully.")
    y_pred_train = y_train  # Just use actual values
    y_pred_test = np.zeros_like(y_test)


STEP 6: MULTI-OUTPUT CLASSIFICATION MODEL
--------------------------------------------------
Features shape: (36, 12)
Targets shape: (36, 3)
Features included: 12 variables
Train size: 28, Test size: 8

Class distribution in training data:
  RuPay_High: Classes [0], Counts [28]
  Operative_High: Classes [0], Counts [28]
  Balance_High: Classes [0], Counts [28]

Training Classifiers...

Multi-output classification cannot be performed meaningfully.


### STEP 7: SATURATION ANALYSIS

In [16]:
print("\nSTEP 7: SATURATION ANALYSIS")
print("-" * 50)

def analyze_saturation(df, penetration_col, utilization_col, thresholds=[200, 250, 300, 350]):
    """Analyze saturation effects at different penetration levels"""
    
    results = []
    
    for threshold in thresholds:
        below = df[df[penetration_col] <= threshold]
        above = df[df[penetration_col] > threshold]
        
        if len(below) > 0 and len(above) > 0:
            mean_util_below = below[utilization_col].mean()
            mean_util_above = above[utilization_col].mean()
            
            # T-test for difference
            try:
                t_stat, p_value = stats.ttest_ind(above[utilization_col], below[utilization_col])
            except:
                t_stat, p_value = 0, 1.0
            
            # Effect size (Cohen's d)
            pooled_var = (below[utilization_col].var() * (len(below) - 1) + 
                         above[utilization_col].var() * (len(above) - 1)) / (len(below) + len(above) - 2)
            pooled_std = np.sqrt(pooled_var) if pooled_var > 0 else 1.0
            cohens_d = (mean_util_above - mean_util_below) / pooled_std if pooled_std > 0 else 0
            
            results.append({
                'threshold': threshold,
                'n_below': len(below),
                'n_above': len(above),
                'mean_below': mean_util_below,
                'mean_above': mean_util_above,
                'difference': mean_util_above - mean_util_below,
                'p_value': p_value,
                'cohens_d': cohens_d
            })
    
    return pd.DataFrame(results)

# Analyze saturation for RuPay penetration
saturation_rupay = analyze_saturation(
    data_rq4,
    'Penetration_Per_1000',
    'RuPay_Penetration'
)

if len(saturation_rupay) > 0:
    print("Saturation Analysis - RuPay Penetration:")
    print(saturation_rupay.to_string(index=False))
else:
    print("Saturation analysis could not be performed - insufficient data distribution")
    saturation_rupay = pd.DataFrame({'cohens_d': [0], 'threshold': [250]})  # Default values

# Analyze saturation for operative rate
saturation_operative = analyze_saturation(
    data_rq4,
    'Penetration_Per_1000',
    'Jan25_Op_Rate'
)

if len(saturation_operative) > 0:
    print("\nSaturation Analysis - Operative Rate:")
    print(saturation_operative.to_string(index=False))
else:
    print("\nSaturation analysis for operative rate could not be performed")
    saturation_operative = pd.DataFrame({'cohens_d': [0], 'threshold': [250]})


STEP 7: SATURATION ANALYSIS
--------------------------------------------------
Saturation analysis could not be performed - insufficient data distribution

Saturation analysis for operative rate could not be performed


### STEP 8: FEATURE IMPORTANCE ANALYSIS

In [18]:
print("\nSTEP 8: FEATURE IMPORTANCE FOR UTILIZATION PREDICTION")
print("-" * 50)

# Check if we have valid targets with multiple classes
if len(valid_targets) > 0 and 0 in valid_targets:  # If RuPay_High has multiple classes
    # Train a Random Forest to get feature importances
    rf_model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
    rf_model.fit(X_train_scaled, y_train[:, 0])  # Predict RuPay_High
    
    # Get feature importances
    importances = rf_model.feature_importances_
    feature_importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    print("Top 10 Feature Importances for RuPay Utilization:")
    print(feature_importance_df.head(10).to_string(index=False))
else:
    # Use correlation-based importance as fallback
    print("Using correlation-based feature importance (due to class imbalance):")
    feature_importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': [abs(pearsonr(X_train[:, i], data_rq4['RuPay_Penetration'][:train_size])[0]) 
                      for i in range(len(feature_names))]
    }).sort_values('importance', ascending=False)
    
    print("Top 10 Feature Correlations with RuPay Utilization:")
    print(feature_importance_df.head(10).to_string(index=False))


STEP 8: FEATURE IMPORTANCE FOR UTILIZATION PREDICTION
--------------------------------------------------
Using correlation-based feature importance (due to class imbalance):
Top 10 Feature Correlations with RuPay Utilization:
                     feature  importance
            Region_Northeast    0.513849
Growth_Operative_Interaction    0.285303
              Growth_2024_25    0.220615
                Region_North    0.219589
                 Region_West    0.213927
               Rural_Percent    0.172936
                 Region_East    0.115888
              Operative_Mean    0.057532
                Region_South    0.044990
        Penetration_Per_1000    0.044724


### STEP 9: CROSS-VALIDATION WITH DIFFERENT THRESHOLDS

In [20]:

print("\nSTEP 9: SENSITIVITY ANALYSIS WITH DIFFERENT THRESHOLDS")
print("-" * 50)

def test_different_thresholds(df, penetration_col, utilization_col):
    """Test correlation at different threshold definitions"""
    
    thresholds = {
        'RuPay': [60, 65, 70, 75, 80],
        'Operative': [70, 75, 80, 85, 90],
        'Penetration': [200, 225, 250, 275, 300]
    }
    
    results = []
    
    for pen_threshold in thresholds['Penetration']:
        for rupay_threshold in thresholds['RuPay']:
            # Create binary variables with these thresholds
            pen_high = (df[penetration_col] > pen_threshold).astype(int)
            rupay_high = (df['RuPay_Penetration'] > rupay_threshold).astype(int)
            
            # Calculate association
            from scipy.stats import chi2_contingency
            contingency = pd.crosstab(pen_high, rupay_high)
            chi2, p_value, _, _ = chi2_contingency(contingency)
            
            # Phi coefficient (effect size for 2x2 table)
            n = len(df)
            phi = np.sqrt(chi2 / n) if n > 0 else 0
            
            results.append({
                'pen_threshold': pen_threshold,
                'rupay_threshold': rupay_threshold,
                'chi2': chi2,
                'p_value': p_value,
                'phi': phi
            })
    
    return pd.DataFrame(results)

# Test sensitivity
sensitivity_results = test_different_thresholds(
    data_rq4,
    'Penetration_Per_1000',
    'RuPay_Penetration'
)

# Show best combinations
best_combos = sensitivity_results.nlargest(5, 'phi')
print("Top 5 Threshold Combinations (by effect size):")
print(best_combos[['pen_threshold', 'rupay_threshold', 'phi', 'p_value']].to_string(index=False))


STEP 9: SENSITIVITY ANALYSIS WITH DIFFERENT THRESHOLDS
--------------------------------------------------
Top 5 Threshold Combinations (by effect size):
 pen_threshold  rupay_threshold  phi  p_value
           200               60  0.0      1.0
           200               65  0.0      1.0
           200               70  0.0      1.0
           200               75  0.0      1.0
           200               80  0.0      1.0


### STEP 10: FINAL SUMMARY AND RECOMMENDATIONS

In [22]:
print("FINAL SUMMARY - RESEARCH QUESTION 4")
print("=" * 80)

print("\n1. CORRELATION FINDINGS:")
print(f"   - Penetration vs RuPay: ρ = {corr_rupay_spearman:.3f}", end="")
if not np.isnan(boot_rupay['ci_lower']):
    print(f" (95% CI: [{boot_rupay['ci_lower']:.3f}, {boot_rupay['ci_upper']:.3f}])")
else:
    print()
print(f"   - Penetration vs Operative: ρ = {corr_op_spearman:.3f}")
print(f"   - Penetration vs Composite: ρ = {corr_comp_spearman:.3f}")

if abs(corr_rupay_spearman) > 0.6:
    print(f" Strong relationship detected (|ρ| > 0.60)")
elif abs(corr_rupay_spearman) > 0.4:
    print(f" Moderate relationship detected (0.40 < |ρ| < 0.60)")
else:
    print(f" Weak relationship detected (|ρ| < 0.40)")

print("\n2. SATURATION INSIGHTS:")
if len(saturation_rupay) > 0 and 'cohens_d' in saturation_rupay.columns:
    optimal_threshold = saturation_rupay.loc[saturation_rupay['cohens_d'].idxmax(), 'threshold']
    print(f"   - Optimal penetration threshold: {optimal_threshold:.0f} per 1000")
    print(f"   - Maximum effect size at this threshold: Cohen's d = {saturation_rupay['cohens_d'].max():.2f}")
else:
    print("   - Saturation analysis could not be completed")

print("\n3. MULTI-OUTPUT MODEL PERFORMANCE:")
if 'valid_targets' in locals() and len(valid_targets) > 0:
    avg_acc = np.mean([accuracy_score(y_test[:, i], y_pred_test[:, i]) 
                      for i in valid_targets if i < len(y_test[0])])
    print(f"   - Average test accuracy (valid targets): {avg_acc:.2f}")
    if 'subset_acc' in locals():
        print(f"   - Subset accuracy (all correct): {subset_acc:.2f}")
    if 'hamming' in locals():
        print(f"   - Hamming loss: {hamming:.3f}")
else:
    print("   - Classification not performed due to class imbalance")

print("\n4. KEY PREDICTORS OF UTILIZATION:")
if 'feature_importance_df' in locals() and len(feature_importance_df) > 0:
    top_3_features = feature_importance_df.head(3)
    for _, row in top_3_features.iterrows():
        print(f"   - {row['feature']}: {row['importance']:.3f}")
else:
    print("   - Feature importance could not be calculated")

print("\n5. POLICY RECOMMENDATIONS:")
if len(saturation_rupay) > 0 and 'cohens_d' in saturation_rupay.columns:
    optimal_threshold = saturation_rupay.loc[saturation_rupay['cohens_d'].idxmax(), 'threshold']
    n_below = len(data_rq4[data_rq4['Penetration_Per_1000'] <= optimal_threshold])
    print(f"   - {n_below} states below optimal threshold ({optimal_threshold:.0f} per 1000) need account opening focus")
    print(f"   - {len(data_rq4) - n_below} states above threshold need utilization campaigns")
else:
    # Use median as default threshold
    median_penetration = data_rq4['Penetration_Per_1000'].median()
    n_below = len(data_rq4[data_rq4['Penetration_Per_1000'] <= median_penetration])
    print(f"   - {n_below} states below median penetration ({median_penetration:.0f} per 1000)")
    print(f"   - {len(data_rq4) - n_below} states above median penetration")

FINAL SUMMARY - RESEARCH QUESTION 4

1. CORRELATION FINDINGS:
   - Penetration vs RuPay: ρ = -0.059 (95% CI: [-0.401, 0.310])
   - Penetration vs Operative: ρ = 0.152
   - Penetration vs Composite: ρ = 0.000
 Weak relationship detected (|ρ| < 0.40)

2. SATURATION INSIGHTS:
   - Optimal penetration threshold: 250 per 1000
   - Maximum effect size at this threshold: Cohen's d = 0.00

3. MULTI-OUTPUT MODEL PERFORMANCE:
   - Classification not performed due to class imbalance

4. KEY PREDICTORS OF UTILIZATION:
   - Region_Northeast: 0.514
   - Growth_Operative_Interaction: 0.285
   - Growth_2024_25: 0.221

5. POLICY RECOMMENDATIONS:
   - 36 states below optimal threshold (250 per 1000) need account opening focus
   - 0 states above threshold need utilization campaigns
